In [2]:
import numpy as np
import pandas as pd 
import networkx as nx
import math
import sympy as sp
import matplotlib.pyplot as plt

from sympy import *
from numpy import linalg as LA
from numpy.linalg import matrix_rank

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ="all"

In [3]:
#Establecer directorio de datos

DIR='/Users/alex/Desktop/MatrizAdjSano/MatrizAdjTumor'
FILE='/adjmatrix_UNT.txt'

file='{}{}'.format(DIR,FILE)

print('File directory:{}'.format(file))



File directory:/Users/alex/Desktop/MatrizAdjSano/MatrizAdjTumor/adjmatrix_UNT.txt


In [4]:
data=pd.read_table(file)

In [5]:
data.shape

(16748, 16748)

In [6]:
a=np.diag(np.ones(16748),0)
A=data-a

In [7]:
#Este modulo calcula un valor umbral ("threshold") que promedia todos los valores de las 
# entradas de un data set

# Función convertir a lista el resultado de aplicar .mean() a un DataFrame

def mean_to_list(df):
    arr=[]
    for i in range(0,len(df)):
        a=df.iloc[i]
        arr.append(a)
    return arr

# Función para calcular el promedio de las entradas de una lista 

def cal_prom_en_list(num):
    sum_num=0
    for entrada in num:
        sum_num=sum_num +entrada
        
    prom = sum_num/len(num)
    return prom

# Cálculo del umbral, promediando sobre todas las entradas de la matriz de adyacencia

A_prom_cols=A.mean()
A_mean_list=mean_to_list(A_prom_cols)
cal_prom_en_list(A_mean_list)
threshold=cal_prom_en_list(A_mean_list)

In [8]:
threshold

0.033064517362779086

In [10]:
#Función para contruir la matriz de grado a partir de la de la matriz de adyacencia

def degreeMat_from_Adj(mat,n):
    new_data=mat.iloc[:n,:n]
    
    new_data2=new_data.apply(lambda x:(np.where(x<threshold,0,x)))
    adj_mat=new_data2.apply(lambda x:(np.where(x>threshold,1,x)))
    
    #completar la matriz triangular superior para tener la forma completa de una matriz
    # simétrica
    
    adj_mod=np.matrix(adj_mat.fillna(0))
    adj_mod_trans=adj_mod.transpose()
    simm_mat=adj_mod+adj_mod_trans    #type(simm_mat)=numpy matrix
    
    
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx) #obtener la Laplaciana asociada a la de adyacencia
    
    L_arr=L_from_Adj.toarray()
    
    L_np=np.matrix(L_arr)
    
    deg_Mat=L_np+simm_mat
    
    return deg_Mat

In [11]:
# El siguiente módulo, crea un array con los primeros "n" genes que deseen obtener del
# Data Frame "data"

#Parametros: data , n 

def get_genes(data,n):
    
    genes_arr = sorted(data) #obtener un array que contiene los elementos del header del DataFrame
    num_genes_extracted = range(n)
    genes = np.array(genes_arr)[num_genes_extracted]
    
    return genes

In [12]:
#La siguiente función sirve para obtener los elementos de la diagonal de la matriz de
# grado y la guarda en un array llamado out_p


def get_elem_in_diag(Mat):
    Mat_mod = np.squeeze(np.asarray(Mat))#convierte numpy matrix en array
    out_p=[]
    for elem in np.nditer(Mat_mod): #nditer itera sobre arrays
        if elem != 0: # omite los valores 0 en el array pues la matriz de grado fuera de la diagonal tiene solo ceros
            out_p.append(int(elem))
            
    return out_p 

In [13]:
#Función para obtener el número mayor de una lista de números dentro de un array

# El parámetro debe tener el mismo número que la dimensión de la matriz 
# de grado que se está analizando

#def get_biggest_num_in_diag(l,n):
#    count=0
#    for i in range(n):
#        for j in range(n):
#            if l[j]>l[i]:
#                count=l[j]
#    return count   

# En la siguiente función el parámetro "a" es un array que contiene los elementos de 
# la diagonal de la matriz de grado y "N" es el número de elementos que se tomará para
# ordenarlos de mayor a menor.

# El output indica la posición de la entrada y no su valor, pues con ello podremos 
# identificar cuál es el gen que tiene más aristas incidentes dentro de él 

def func(a,N):
    return np.argsort(a)[::-1][:N]

In [14]:
func(get_elem_in_diag(degreeMat_from_Adj(A,100)),100)

array([24, 61, 83,  6, 60, 31, 86, 14, 47, 79, 53, 90, 39, 92, 44, 94, 82,
       34, 57, 64, 17, 97, 13, 36, 29, 77, 78, 27,  3, 41, 32, 18, 48,  0,
       87, 72,  4, 10, 12, 75, 38, 95, 46, 21, 22, 85, 45, 59, 28, 42, 11,
       96, 43, 81,  9, 68, 91, 56, 71, 69, 51, 37, 66, 70, 62, 15, 23, 30,
       99, 33, 98, 84, 58, 16, 55, 89, 49,  8, 20,  1, 63, 50, 93, 74, 73,
        7, 76,  5, 88, 65, 52, 25, 40, 80, 19, 26, 54, 67,  2, 35])

In [15]:
func(get_elem_in_diag(degreeMat_from_Adj(A,1000)),1000)

array([465, 221, 555, 365, 732, 373, 346, 514,  66, 166, 557, 358, 251,
       214, 649, 894, 317,  41, 811, 375, 918, 263, 640, 806, 153, 619,
        31, 565, 297, 127,  43, 520, 728, 241, 690, 747, 371, 398,  47,
       820, 316, 197,  94, 652, 156,  61, 455, 788, 123, 171, 161, 710,
        79, 763, 571, 289, 198, 586, 258, 863, 424, 639, 752, 205, 545,
       415, 892, 641,   3, 165, 441, 862, 479, 868, 132, 593, 861, 549,
       734, 389, 716, 803, 874, 799, 204, 845,  74, 217, 357, 667, 147,
        29, 815, 929, 839, 329, 943, 920, 288,  83, 150, 661, 588,  75,
       886, 883, 805, 272, 416, 426, 663, 534, 470, 266, 719, 891, 481,
       843, 847,  53, 781, 162, 852, 896, 574, 611, 249, 818, 561, 253,
         0, 889, 160, 438, 991, 973, 448, 286,  82, 701, 237, 628, 854,
       875, 392, 767, 445, 882,  24,  48, 633, 212, 718, 686, 469, 798,
       341, 990, 695, 927, 254,  71, 459, 961, 271, 944, 169, 864, 804,
       427, 853, 616,  12, 617, 490, 282, 758, 179, 871, 432, 13

In [16]:
func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

array([ 465, 1716,  365, ..., 1910, 1847,  414])

In [17]:
func(get_elem_in_diag(degreeMat_from_Adj(A,3000)),3000)

array([ 465, 2457, 2005, ..., 2112, 2473,  414])

In [18]:
func(get_elem_in_diag(degreeMat_from_Adj(A,4000)),4000)

array([ 465, 3742,  365, ...,  414, 1910, 2112])

In [19]:
func(get_elem_in_diag(degreeMat_from_Adj(A,5000)),5000)

array([ 465, 3742,  365, ...,  414, 1910, 2112])

In [20]:
func(get_elem_in_diag(degreeMat_from_Adj(A,6000)),6000)

array([ 465, 3742,  365, ..., 2112, 5860, 5143])

In [21]:
func(get_elem_in_diag(degreeMat_from_Adj(A,7000)),7000)

array([ 465, 3742,  365, ..., 5860, 2112, 5143])

In [22]:
func(get_elem_in_diag(degreeMat_from_Adj(A,8000)),8000)

array([ 465, 7948, 3742, ..., 2112, 7635, 5143])

In [23]:
func(get_elem_in_diag(degreeMat_from_Adj(A,9000)),9000)

array([ 465, 7948, 3742, ..., 2112, 5143, 8300])

In [24]:
func(get_elem_in_diag(degreeMat_from_Adj(A,10000)),10000)

array([ 465, 7948, 7261, ..., 5143, 9116, 8300])

In [13]:
get_genes(data,16748)[24]

'ENSG00000003056'

In [14]:
A.mean()

ENSG00000000003    0.000000
ENSG00000000419    0.011007
ENSG00000000457    0.021131
ENSG00000000460    0.064793
ENSG00000000938    0.029265
                     ...   
hsa-mir-9500       0.028380
hsa-mir-96         0.034278
hsa-mir-98         0.031001
hsa-mir-99a        0.032875
hsa-mir-99b        0.032385
Length: 16748, dtype: float64

In [15]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

array([ 465, 1716,  365, ..., 1910, 1847,  414])

In [17]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,100)),100)

array([24, 61, 83,  6, 60, 31, 86, 14, 47, 79, 53, 90, 39, 92, 44, 94, 82,
       34, 57, 64, 17, 97, 13, 36, 29, 77, 78, 27,  3, 41, 32, 18, 48,  0,
       87, 72,  4, 10, 12, 75, 38, 95, 46, 21, 22, 85, 45, 59, 28, 42, 11,
       96, 43, 81,  9, 68, 91, 56, 71, 69, 51, 37, 66, 70, 62, 15, 23, 30,
       99, 33, 98, 84, 58, 16, 55, 89, 49,  8, 20,  1, 63, 50, 93, 74, 73,
        7, 76,  5, 88, 65, 52, 25, 40, 80, 19, 26, 54, 67,  2, 35])

In [ ]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,9000)),9000)

In [1]:
 func(get_elem_in_diag(degreeMat_from_Adj(A,2000)),2000)

NameError: name 'func' is not defined